# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv from part 1
file = os.path.join('..', 'output_data', 'cities.csv')

# Read and display the dataframe
cities_df = pd.read_csv(file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,shirokiy,49.7580,129.5230,-14.19,87,2,6.78,RU,1612664265
1,faanui,-16.4833,-151.7500,83.08,65,0,10.07,PF,1612664265
2,yellowknife,62.4560,-114.3525,-41.80,74,1,3.44,CA,1612663989
3,rikitea,-23.1203,-134.9692,78.85,65,100,10.00,PF,1612664007
4,upernavik,72.7868,-56.1549,18.57,76,28,3.71,GL,1612664265


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Set locations with lat and long from cities DataFrame
locations = cities_df[['Lat', 'Lng']]

#Set weight for gmap layer (humidity)
humidity = cities_df['Humidity']

# Plot gmaps with given info
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,
                                 max_intensity=100, point_radius=2
                                 )

# Add heat layer to gmaps
fig.add_layer(heat_layer)

#display gmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Find cities with max temps between 70 and 80
ideal_con = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70), :]

# Add to existing ideal DataFrame and find wind speeds below 10 mph
ideal_con = ideal_con.loc[ideal_con['Wind Speed'] < 10]

# Keep adding to the ideal DataFrame and find locations with zero cloudiness
ideal_con = ideal_con.loc[ideal_con['Cloudiness'] == 0]

# Display ideal DataFrame
ideal_con

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,gambela,8.2500,34.5833,72.27,22,0,2.06,ET,1612664267
32,macau,22.2006,113.5461,75.20,83,0,3.44,MO,1612664269
97,la orilla,17.9833,-102.2333,73.11,77,0,1.90,MX,1612664203
117,young,-32.6833,-57.6333,75.00,68,0,4.79,UY,1612664280
158,bambous virieux,-20.3428,57.7575,75.00,86,0,1.99,MU,1612664011
162,vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612664286
195,damara,4.9608,18.7035,75.20,94,0,2.68,CF,1612664290
196,santiago del estero,-27.7951,-64.2615,75.99,54,0,4.00,AR,1612664287
222,general roca,-39.0333,-67.5833,75.20,69,0,2.30,AR,1612664153
299,juazeirinho,-6.8167,-38.0500,77.16,68,0,7.29,BR,1612664305


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store ideal conditions df into new variable and add new column
hotel_df = ideal_con
hotel_df['Hotel Name'] = ''

# Display new dataframe with added column
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,gambela,8.2500,34.5833,72.27,22,0,2.06,ET,1612664267,
32,macau,22.2006,113.5461,75.20,83,0,3.44,MO,1612664269,
97,la orilla,17.9833,-102.2333,73.11,77,0,1.90,MX,1612664203,
117,young,-32.6833,-57.6333,75.00,68,0,4.79,UY,1612664280,
158,bambous virieux,-20.3428,57.7575,75.00,86,0,1.99,MU,1612664011,
162,vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612664286,
195,damara,4.9608,18.7035,75.20,94,0,2.68,CF,1612664290,
196,santiago del estero,-27.7951,-64.2615,75.99,54,0,4.00,AR,1612664287,
222,general roca,-39.0333,-67.5833,75.20,69,0,2.30,AR,1612664153,
299,juazeirinho,-6.8167,-38.0500,77.16,68,0,7.29,BR,1612664305,


In [ ]:
# Google maps example query search for reference when making params
# https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY

In [6]:
# Create hotel names to add to and set query params
hotel_names = []
params = {"radius": 5000, 
          "type": "hotel",
          "keyword": "hotel",          
          "key": g_key}
print('Start of Hotel search..')
print('---------------------------------')
# Loop through lat and long with iterrows to find nearest hotel
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    
    #set base url and response
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(url, params=params).json()
    
    #extract the results from API response
    results = response['results']
    
    try:
        hotel_names.append(results[0]['name'])
        print(f'The closest hotel at index {index} is: {results[0]["name"]}')
    except(IndexError):
        hotel_names.append('')
        print(f'Hotel was not found.. at index {index}: {lat}, {long}')
print('---------------------------------')
print('End of search..')

Start of Hotel search..
---------------------------------
The closest hotel at index 19 is: Grand Resort And Spa,Gambella
The closest hotel at index 32 is: The St. Regis Zhuhai
The closest hotel at index 97 is: City Express Lázaro Cárdenas
The closest hotel at index 117 is: Hotel Young
Hotel was not found.. at index 158: -20.3428, 57.7575
The closest hotel at index 162 is: OURE TERA beach resort
Hotel was not found.. at index 195: 4.9608, 18.7035
The closest hotel at index 196 is: Hotel Altos del Estero
The closest hotel at index 222 is: Estepa Hotel
The closest hotel at index 299 is: Pousada Noah3
The closest hotel at index 357 is: Lodge Roche Tamarin & Spa Village Nature
The closest hotel at index 403 is: Lakefront Lodge Te Anau
The closest hotel at index 457 is: Hotel de Casablanca
The closest hotel at index 509 is: Swe Tha Har - Guest House
The closest hotel at index 565 is: The Palm House Hotel
The closest hotel at index 602 is: Praia Hotel Imbituba
-------------------------------

In [7]:
# Add results from API response to hotel DataFrame..
hotel_df['Hotel Name'] = hotel_names
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,gambela,8.2500,34.5833,72.27,22,0,2.06,ET,1612664267,"Grand Resort And Spa,Gambella"
32,macau,22.2006,113.5461,75.20,83,0,3.44,MO,1612664269,The St. Regis Zhuhai
97,la orilla,17.9833,-102.2333,73.11,77,0,1.90,MX,1612664203,City Express Lázaro Cárdenas
117,young,-32.6833,-57.6333,75.00,68,0,4.79,UY,1612664280,Hotel Young
158,bambous virieux,-20.3428,57.7575,75.00,86,0,1.99,MU,1612664011,
162,vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612664286,OURE TERA beach resort
195,damara,4.9608,18.7035,75.20,94,0,2.68,CF,1612664290,
196,santiago del estero,-27.7951,-64.2615,75.99,54,0,4.00,AR,1612664287,Hotel Altos del Estero
222,general roca,-39.0333,-67.5833,75.20,69,0,2.30,AR,1612664153,Estepa Hotel
299,juazeirinho,-6.8167,-38.0500,77.16,68,0,7.29,BR,1612664305,Pousada Noah3


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))